In [17]:
"""
CELL 1: SETUP
Description: Import required libraries, load configuration, and load prompts
"""

# Load configuration
%run vyapar_config.ipynb

# Load prompts library
%run vaani_prompts.ipynb

# Additional imports for AI functions
import openai
from anthropic import Anthropic
import time
from typing import List, Dict, Tuple, Optional

print("✅ vaani_functions.ipynb loaded")
print("✅ Config available via get_item(), get_api_key(), etc.")
print("✅ Prompts library loaded")

✅ Libraries imported successfully
✅ Master Registry Link configured
📍 Link: https://docs.google.com/spreadsheets/d/e/2PACX-1vQdOVYDNLuMG...

🔄 Testing registry load...
✅ Registry loaded successfully: 8 items found

📋 Available items in registry:
  1. default_model
  2. openai_api_key
  3. anthropic_api_key
  4. master_registry_link
  5. usage_data
  6. dropoff_analysis
  7. user_research
  8. excel_db_1000_items

🧪 Testing get_item() function:

1. Testing with 'default_model':
✅ Registry loaded successfully: 8 items found
✅ Retrieved 'default_model'
   Result: {'model': 'claude-sonnet-4-20250514', 'temperature': 0.3, 'max_tokens': 1000, 'provider': 'anthropic'}

2. Testing with non-existent item:
✅ Registry loaded successfully: 8 items found
❌ Item 'this_does_not_exist' not found in registry
💡 Available items: default_model, openai_api_key, anthropic_api_key, master_registry_link, usage_data, dropoff_analysis, user_research, excel_db_1000_items

3. Testing with Sarvam API (might not ha

In [9]:
"""
CELL 2: OPENAI CLIENT SETUP
Description: Create OpenAI client using API key from registry
"""

def get_openai_client():
    """
    Initialize and return OpenAI client.
    Uses API key from registry.
    """
    api_key = get_api_key('openai_api_key')
    
    if not api_key or 'YOUR' in api_key:
        print("⚠️ OpenAI API key not configured properly")
        return None
    
    client = openai.OpenAI(api_key=api_key)
    print("✅ OpenAI client initialized")
    return client

# Test it
openai_client = get_openai_client()

✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly


In [10]:
"""
CELL 3: ANTHROPIC CLIENT SETUP
Description: Create Anthropic client using API key from registry
"""

def get_anthropic_client():
    """
    Initialize and return Anthropic client.
    Uses API key from registry.
    """
    api_key = get_api_key('anthropic_api_key')
    
    if not api_key or 'YOUR' in api_key:
        print("⚠️ Anthropic API key not configured properly")
        return None
    
    client = Anthropic(api_key=api_key)
    print("✅ Anthropic client initialized")
    return client

# Test it
anthropic_client = get_anthropic_client()

✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized


In [11]:
"""
CELL 4: AI AGENT 1 - BASIC EXTRACTOR
Description: Extracts amount, item name, and category from voice input
Use case: Quick, simple extractions
"""

def agent_basic_extractor(voice_input: str, model: str = None) -> Dict:
    """
    Basic extraction agent: Extracts amount, item, category from voice input.
    
    Args:
        voice_input: The voice transcription (e.g., "chai samosa 140 rupees")
        model: Model to use (defaults to registry default_model)
    
    Returns:
        {
            'amount': 140,
            'item': 'Chai Samosa',
            'category': 'Food',
            'raw_response': '...',
            'model_used': 'gpt-4o-mini',
            'time_taken': 1.2
        }
    """
    # Get model config
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    # Create prompt
    prompt = f"""You are an expense tracking assistant. Extract the following from the voice input:

Voice Input: "{voice_input}"

Extract and return ONLY a JSON object with these fields:
- amount: The amount spent (number only, no currency)
- item: The item name
- category: Best category for this expense (Food, Transport, Utilities, etc.)

Return ONLY valid JSON, nothing else."""

    # Call LLM
    start_time = time.time()
    
    try:
        if 'gpt' in model:
            # OpenAI call
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            # Anthropic call
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        # Parse JSON response
        import json
        extracted = json.loads(raw_response)
        
        # Add metadata
        extracted['raw_response'] = raw_response
        extracted['model_used'] = model
        extracted['time_taken'] = round(time_taken, 2)
        
        return extracted
    
    except Exception as e:
        return {
            'error': str(e),
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

# Test it
print("🧪 Testing Basic Extractor Agent:")
test_result = agent_basic_extractor("chai samosa 140 rupees")
print(test_result)

🧪 Testing Basic Extractor Agent:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
{'error': 'OpenAI client not available'}


In [12]:
"""
CELL 5: AI AGENT 2 - SMART CATEGORIZER
Description: Uses context from Excel DB for more accurate categorization
Use case: When you want high accuracy categorization
"""

def agent_smart_categorizer(voice_input: str, excel_db_context: str = None, model: str = None) -> Dict:
    """
    Smart categorization agent: Better category matching using Excel DB context.
    
    Args:
        voice_input: The voice transcription
        excel_db_context: Sample categories from Excel DB (optional)
        model: Model to use (defaults to registry default_model)
    
    Returns:
        Same as agent_basic_extractor but with improved categorization
    """
    # Get model config
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    # Add context to prompt if provided
    context_section = ""
    if excel_db_context:
        context_section = f"\n\nCommon categories used by similar businesses:\n{excel_db_context}\n"
    
    # Create enhanced prompt
    prompt = f"""You are an expense tracking assistant for Indian MSMEs. Extract information from this voice input.

Voice Input: "{voice_input}"
{context_section}
Extract and return ONLY a JSON object:
- amount: Amount spent (number only)
- item: Item name
- category: Most appropriate category (consider common Indian business expenses)

Return ONLY valid JSON."""

    # Call LLM (same logic as basic extractor)
    start_time = time.time()
    
    try:
        if 'gpt' in model:
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        # Parse JSON
        import json
        extracted = json.loads(raw_response)
        
        # Add metadata
        extracted['raw_response'] = raw_response
        extracted['model_used'] = model
        extracted['time_taken'] = round(time_taken, 2)
        extracted['used_context'] = excel_db_context is not None
        
        return extracted
    
    except Exception as e:
        return {
            'error': str(e),
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

# Test it
print("🧪 Testing Smart Categorizer Agent:")
test_result = agent_smart_categorizer("petrol 500 rupees")
print(test_result)

🧪 Testing Smart Categorizer Agent:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
{'error': 'OpenAI client not available'}


In [13]:
"""
CELL 6: AI AGENT 3 - MULTI-ITEM HANDLER
Description: Handles voice inputs with multiple items
Use case: "chai 60 rupees, samosa 80 rupees"
"""

def agent_multi_item_handler(voice_input: str, model: str = None) -> Dict:
    """
    Multi-item handler: Extracts multiple items from single voice input.
    
    Args:
        voice_input: Voice input with multiple items
        model: Model to use
    
    Returns:
        {
            'items': [
                {'amount': 60, 'item': 'Chai', 'category': 'Food'},
                {'amount': 80, 'item': 'Samosa', 'category': 'Food'}
            ],
            'total_amount': 140,
            'item_count': 2,
            'model_used': 'gpt-4o-mini',
            'time_taken': 1.5
        }
    """
    # Get model config
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    # Create prompt
    prompt = f"""You are an expense tracking assistant. The voice input may contain multiple items.

Voice Input: "{voice_input}"

Extract ALL items mentioned. Return ONLY a JSON object:
{{
    "items": [
        {{"amount": number, "item": "name", "category": "category"}},
        ...
    ],
    "total_amount": sum of all amounts
}}

Return ONLY valid JSON."""

    start_time = time.time()
    
    try:
        if 'gpt' in model:
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 800)
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 800)
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        # Parse JSON
        import json
        extracted = json.loads(raw_response)
        
        # Add metadata
        extracted['item_count'] = len(extracted.get('items', []))
        extracted['raw_response'] = raw_response
        extracted['model_used'] = model
        extracted['time_taken'] = round(time_taken, 2)
        
        return extracted
    
    except Exception as e:
        return {
            'error': str(e),
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

# Test it
print("🧪 Testing Multi-Item Handler Agent:")
test_result = agent_multi_item_handler("chai 60 rupees, samosa 80 rupees")
print(test_result)

🧪 Testing Multi-Item Handler Agent:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
{'error': 'OpenAI client not available'}


In [14]:
"""
CELL 7: AGENT ROUTER
Description: Intelligently routes to the right agent based on input complexity
"""

def route_to_agent(voice_input: str, model: str = None) -> Dict:
    """
    Smart router: Analyzes input and calls the appropriate agent.
    
    Logic:
    - If multiple items detected → Multi-Item Handler
    - If simple input → Basic Extractor
    - If needs context → Smart Categorizer
    
    Args:
        voice_input: The voice input
        model: Model to use (optional)
    
    Returns:
        Result from the chosen agent + routing metadata
    """
    # Simple heuristics for routing
    voice_lower = voice_input.lower()
    
    # Check for multiple items (commas, "and", multiple numbers)
    has_comma = ',' in voice_input
    has_and = ' and ' in voice_lower or ' aur ' in voice_lower
    number_count = sum(c.isdigit() for c in voice_input)
    
    if has_comma or has_and or number_count > 6:
        # Route to multi-item handler
        print("🎯 Routing to: Multi-Item Handler")
        result = agent_multi_item_handler(voice_input, model)
        result['agent_used'] = 'multi_item_handler'
    
    else:
        # Route to basic extractor
        print("🎯 Routing to: Basic Extractor")
        result = agent_basic_extractor(voice_input, model)
        result['agent_used'] = 'basic_extractor'
    
    return result

# Test it
print("\n🧪 Testing Agent Router:")
print("\n1. Single item:")
result1 = route_to_agent("chai 60 rupees")
print(f"Result: {result1}\n")

print("2. Multiple items:")
result2 = route_to_agent("chai 60 rupees, samosa 80 rupees")
print(f"Result: {result2}")


🧪 Testing Agent Router:

1. Single item:
🎯 Routing to: Basic Extractor
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
Result: {'error': 'OpenAI client not available', 'agent_used': 'basic_extractor'}

2. Multiple items:
🎯 Routing to: Multi-Item Handler
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
⚠️ OpenAI API key not configured properly
Result: {'error': 'OpenAI client not available', 'agent_used': 'multi_item_handler'}


In [15]:
"""
CELL 8: UTILITY FUNCTIONS
Description: Helper functions for formatting, validation, etc.
"""

def format_amount(amount: any) -> float:
    """Convert amount to float, handle different formats"""
    try:
        if isinstance(amount, str):
            # Remove currency symbols, commas
            amount = amount.replace('₹', '').replace(',', '').strip()
        return float(amount)
    except:
        return None

def validate_extraction(result: Dict) -> Tuple[bool, List[str]]:
    """
    Validate extraction result.
    Returns: (is_valid, list_of_errors)
    """
    errors = []
    
    if 'error' in result:
        errors.append(f"API Error: {result['error']}")
        return False, errors
    
    # Check required fields
    if 'amount' not in result:
        errors.append("Missing amount")
    elif not isinstance(result['amount'], (int, float)):
        errors.append("Amount is not a number")
    
    if 'item' not in result:
        errors.append("Missing item name")
    elif not result['item']:
        errors.append("Item name is empty")
    
    return len(errors) == 0, errors

# Test utilities
print("🧪 Testing Utility Functions:")
print(f"Format '₹1,500': {format_amount('₹1,500')}")
print(f"Validate good result: {validate_extraction({'amount': 100, 'item': 'Chai'})}")
print(f"Validate bad result: {validate_extraction({'error': 'Failed'})}")

🧪 Testing Utility Functions:
Format '₹1,500': 1500.0
Validate good result: (True, [])
Validate bad result: (False, ['API Error: Failed'])


In [16]:
"""
CELL 9: SUMMARY
Description: Shows all available functions in this notebook
"""

print("""
📚 VAANI FUNCTIONS AVAILABLE
=====================================================

🤖 AI AGENTS:
1. agent_basic_extractor(voice_input, model=None)
   - Simple, fast extraction
   
2. agent_smart_categorizer(voice_input, excel_db_context=None, model=None)
   - Better categorization with context
   
3. agent_multi_item_handler(voice_input, model=None)
   - Handles multiple items in one input
   
4. route_to_agent(voice_input, model=None)
   - Smart router (RECOMMENDED)

🛠️ UTILITIES:
- format_amount(amount) - Clean amount formatting
- validate_extraction(result) - Validate extraction results

🔌 CLIENTS:
- get_openai_client() - Get OpenAI client
- get_anthropic_client() - Get Anthropic client

=====================================================
✅ All functions loaded and ready!
""")


📚 VAANI FUNCTIONS AVAILABLE

🤖 AI AGENTS:
1. agent_basic_extractor(voice_input, model=None)
   - Simple, fast extraction
   
2. agent_smart_categorizer(voice_input, excel_db_context=None, model=None)
   - Better categorization with context
   
3. agent_multi_item_handler(voice_input, model=None)
   - Handles multiple items in one input
   
4. route_to_agent(voice_input, model=None)
   - Smart router (RECOMMENDED)

🛠️ UTILITIES:
- format_amount(amount) - Clean amount formatting
- validate_extraction(result) - Validate extraction results

🔌 CLIENTS:
- get_openai_client() - Get OpenAI client
- get_anthropic_client() - Get Anthropic client

✅ All functions loaded and ready!



In [22]:
"""
CELL 9B: JSON EXTRACTION HELPER
Description: Safely extract JSON from LLM responses that may have markdown or extra text
"""

import re

def extract_json_from_response(response_text: str) -> str:
    """
    Extract JSON from LLM response, handling markdown code blocks and extra text.
    
    Args:
        response_text: Raw response from LLM
    
    Returns:
        Clean JSON string
    """
    # Remove markdown code blocks
    response_text = re.sub(r'```json\s*', '', response_text)
    response_text = re.sub(r'```\s*', '', response_text)
    
    # Try to find JSON object in the text
    # Look for { ... } pattern
    json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
    
    if json_match:
        return json_match.group(0)
    
    # If no JSON found, return original
    return response_text.strip()

# Test it
print("🧪 Testing JSON Extractor:")
test_cases = [
    '{"amount": 140}',  # Plain JSON
    '```json\n{"amount": 140}\n```',  # Markdown
    'Here is the result:\n{"amount": 140}',  # With prefix
    '{"amount": 140}\nThis is the expense.',  # With suffix
]

for test in test_cases:
    extracted = extract_json_from_response(test)
    print(f"\nInput: {test[:50]}...")
    print(f"Extracted: {extracted[:50]}...")

🧪 Testing JSON Extractor:

Input: {"amount": 140}...
Extracted: {"amount": 140}...

Input: ```json
{"amount": 140}
```...
Extracted: {"amount": 140}...

Input: Here is the result:
{"amount": 140}...
Extracted: {"amount": 140}...

Input: {"amount": 140}
This is the expense....
Extracted: {"amount": 140}...


In [23]:
"""
CELL 10: INTENT DETECTION AGENT (UPDATED)
Description: Identifies if input is relevant and classifies transaction type
Uses: PROMPT_INTENT_DETECTION from prompts library
"""

def agent_intent_detector(text_input: str, model: str = None) -> Dict:
    """
    Detects if input is relevant to VAANI and identifies transaction type.
    
    Args:
        text_input: User's text input
        model: Model to use (defaults to registry default_model)
    
    Returns:
        {
            'is_relevant': True/False,
            'transaction_type': 'expense'/'sale'/'purchase'/etc.,
            'confidence': 0.0-1.0,
            'reason': 'explanation',
            'model_used': 'model_name',
            'time_taken': 1.2
        }
    """
    # Get model config
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    # Get prompt from library
    prompt = get_prompt('intent', text_input=text_input)
    
    start_time = time.time()
    
    try:
        if 'gpt' in model:
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
                max_tokens=200
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
                max_tokens=200
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        # Extract and parse JSON
        import json
        clean_json = extract_json_from_response(raw_response)
        intent_result = json.loads(clean_json)
        
        # Add metadata
        intent_result['raw_response'] = raw_response
        intent_result['model_used'] = model
        intent_result['time_taken'] = round(time_taken, 2)
        
        return intent_result
    
    except json.JSONDecodeError as e:
        return {
            'error': f'JSON parsing failed: {str(e)}',
            'raw_response': raw_response if 'raw_response' in locals() else None,
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }
    
    except Exception as e:
        return {
            'error': str(e),
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

# Test it
print("🧪 Testing Intent Detection Agent:")
test_cases = [
    "chai samosa 140 rupees",
    "what's the weather today?",
    "Sharma ji paid 5000 rupees",
]

for test in test_cases:
    result = agent_intent_detector(test)
    print(f"\nInput: '{test}'")
    if 'error' not in result:
        print(f"  Relevant: {result.get('is_relevant')}")
        print(f"  Type: {result.get('transaction_type')}")
        print(f"  Confidence: {result.get('confidence')}")
    else:
        print(f"  Error: {result['error']}")

🧪 Testing Intent Detection Agent:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

Input: 'chai samosa 140 rupees'
  Relevant: True
  Type: expense
  Confidence: 0.85
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

Input: 'what's the weather today?'
  Relevant: False
  Type: not_relevant
  Confidence: 1.0
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

Input: 'Sharma ji paid 5000 rupees'
  Relevant: True
  Type: payment_in
  Confidence: 0.9


In [24]:
"""
CELL 11: TRANSACTION-SPECIFIC EXTRACTION AGENT (UPDATED)
Description: Extracts data using the appropriate transaction-type prompt
Uses: PROMPT_EXPENSE_EXTRACTION, PROMPT_SALE_EXTRACTION, etc.
"""

def agent_transaction_extractor(text_input: str, transaction_type: str, model: str = None) -> Dict:
    """
    Extracts transaction details using the appropriate prompt for the transaction type.
    
    Args:
        text_input: User's text input
        transaction_type: Type of transaction (expense/sale/purchase/payment_in/payment_out)
        model: Model to use (defaults to registry default)
    
    Returns:
        Extracted transaction data as dictionary
    """
    # Get model config
    if model is None:
        model_config = get_model_config()
        model = model_config['model']
    else:
        model_config = get_model_config(model)
    
    # Get appropriate prompt from library
    prompt = get_prompt(transaction_type, text_input=text_input)
    
    start_time = time.time()
    
    try:
        if 'gpt' in model:
            client = get_openai_client()
            if not client:
                return {'error': 'OpenAI client not available'}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.choices[0].message.content
        
        elif 'claude' in model:
            client = get_anthropic_client()
            if not client:
                return {'error': 'Anthropic client not available'}
            
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=model_config.get('temperature', 0.3),
                max_tokens=model_config.get('max_tokens', 500)
            )
            raw_response = response.content[0].text
        
        else:
            return {'error': f'Unknown model: {model}'}
        
        time_taken = time.time() - start_time
        
        # Extract and parse JSON
        import json
        clean_json = extract_json_from_response(raw_response)
        extracted = json.loads(clean_json)
        
        # Add metadata
        extracted['transaction_type'] = transaction_type
        extracted['raw_response'] = raw_response
        extracted['model_used'] = model
        extracted['time_taken'] = round(time_taken, 2)
        
        return extracted
    
    except json.JSONDecodeError as e:
        return {
            'error': f'JSON parsing failed: {str(e)}',
            'raw_response': raw_response if 'raw_response' in locals() else None,
            'transaction_type': transaction_type,
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }
    
    except Exception as e:
        return {
            'error': str(e),
            'transaction_type': transaction_type,
            'model_used': model,
            'time_taken': round(time.time() - start_time, 2)
        }

# Test it
print("🧪 Testing Transaction Extraction Agent:")
print("\n1. Expense:")
result1 = agent_transaction_extractor("chai samosa 140 rupees", "expense")
if 'error' not in result1:
    print(f"   Amount: {result1.get('amount')}, Item: {result1.get('item')}, Category: {result1.get('category')}")
else:
    print(f"   Error: {result1['error']}")

print("\n2. Sale:")
result2 = agent_transaction_extractor("Sharma ji bought 5kg rice for 250 rupees", "sale")
if 'error' not in result2:
    print(f"   Customer: {result2.get('customer_name')}, Amount: {result2.get('amount')}")
else:
    print(f"   Error: {result2['error']}")

🧪 Testing Transaction Extraction Agent:

1. Expense:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
   Amount: 140, Item: chai and samosa, Category: Food & Beverages

2. Sale:
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized
   Customer: Sharma ji, Amount: 250


In [20]:
"""
CELL 12: SMART ROUTER WITH INTENT DETECTION
Description: Complete flow - Intent detection → Transaction extraction
"""

def route_with_intent(text_input: str, transaction_type: str = None, model: str = None) -> Dict:
    """
    Smart router with intent detection.
    
    Flow:
    1. If transaction_type provided → Skip intent detection, go directly to extraction
    2. If transaction_type NOT provided → Run intent detection first, then extract
    
    Args:
        text_input: User's text input
        transaction_type: Optional - if known, skip intent detection
        model: Model to use
    
    Returns:
        Complete result with intent + extraction data
    """
    result = {
        'input': text_input,
        'intent_detection_skipped': transaction_type is not None
    }
    
    # Step 1: Intent Detection (if needed)
    if transaction_type is None:
        print("🔍 Running intent detection...")
        intent_result = agent_intent_detector(text_input, model)
        
        if 'error' in intent_result:
            return {'error': f"Intent detection failed: {intent_result['error']}"}
        
        result['intent'] = intent_result
        
        # Check if relevant
        if not intent_result.get('is_relevant'):
            result['status'] = 'not_relevant'
            result['message'] = "Input is not relevant for business transactions"
            return result
        
        # Get transaction type from intent
        transaction_type = intent_result.get('transaction_type')
        
        if transaction_type == 'not_relevant':
            result['status'] = 'not_relevant'
            result['message'] = intent_result.get('reason')
            return result
    
    else:
        print(f"⏭️  Skipping intent detection, using: {transaction_type}")
        result['intent'] = {'transaction_type': transaction_type, 'skipped': True}
    
    # Step 2: Transaction-specific extraction
    print(f"📊 Extracting {transaction_type} data...")
    extraction_result = agent_transaction_extractor(text_input, transaction_type, model)
    
    if 'error' in extraction_result:
        result['error'] = f"Extraction failed: {extraction_result['error']}"
        return result
    
    result['extraction'] = extraction_result
    result['status'] = 'success'
    result['transaction_type'] = transaction_type
    
    return result

# Test it
print("🧪 Testing Smart Router:")

print("\n" + "="*60)
print("TEST 1: With Intent Detection (no transaction_type provided)")
print("="*60)
result1 = route_with_intent("chai samosa 140 rupees")
print(f"\nResult: {result1.get('status')}")
if result1.get('status') == 'success':
    print(f"Transaction Type: {result1.get('transaction_type')}")
    print(f"Extracted Data: {result1['extraction']}")

print("\n" + "="*60)
print("TEST 2: Skip Intent Detection (transaction_type='expense')")
print("="*60)
result2 = route_with_intent("petrol 500 rupees", transaction_type='expense')
print(f"\nResult: {result2.get('status')}")
print(f"Intent Skipped: {result2.get('intent_detection_skipped')}")
if result2.get('status') == 'success':
    print(f"Extracted Data: {result2['extraction']}")

print("\n" + "="*60)
print("TEST 3: Not Relevant Input")
print("="*60)
result3 = route_with_intent("what's the weather today?")
print(f"\nResult: {result3.get('status')}")
print(f"Message: {result3.get('message')}")

🧪 Testing Smart Router:

TEST 1: With Intent Detection (no transaction_type provided)
🔍 Running intent detection...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

Result: None

TEST 2: Skip Intent Detection (transaction_type='expense')
⏭️  Skipping intent detection, using: expense
📊 Extracting expense data...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

Result: None
Intent Skipped: True

TEST 3: Not Relevant Input
🔍 Running intent detection...
✅ Registry loaded successfully: 8 items found
✅ Registry loaded successfully: 8 items found
✅ Anthropic client initialized

Result: None
Message: None


In [21]:
"""
CELL 13: FUNCTIONS SUMMARY
Description: Shows all available functions with new intent-based flow
"""

print("""
📚 VAANI FUNCTIONS - UPDATED
=====================================================

🎯 MAIN ROUTER (RECOMMENDED):
- route_with_intent(text_input, transaction_type=None, model=None)
  → Complete flow with intent detection
  → Use transaction_type to skip intent detection

🔍 INTENT DETECTION:
- agent_intent_detector(text_input, model=None)
  → Classify transaction type

📊 TRANSACTION EXTRACTION:
- agent_transaction_extractor(text_input, transaction_type, model=None)
  → Extract data using transaction-specific prompt

🤖 LEGACY AGENTS (still available):
- agent_basic_extractor()
- agent_smart_categorizer()
- agent_multi_item_handler()

🛠️ UTILITIES:
- format_amount()
- validate_extraction()

USAGE EXAMPLES:
---------------
# Full flow with intent detection
result = route_with_intent("chai 60 rupees")

# Skip intent if you know the type
result = route_with_intent("petrol 500", transaction_type='expense')

# Just intent detection
intent = agent_intent_detector("chai 60 rupees")

# Just extraction
data = agent_transaction_extractor("chai 60", "expense")

=====================================================
✅ Functions ready! Use route_with_intent() for most cases
""")


📚 VAANI FUNCTIONS - UPDATED

🎯 MAIN ROUTER (RECOMMENDED):
- route_with_intent(text_input, transaction_type=None, model=None)
  → Complete flow with intent detection
  → Use transaction_type to skip intent detection

🔍 INTENT DETECTION:
- agent_intent_detector(text_input, model=None)
  → Classify transaction type

📊 TRANSACTION EXTRACTION:
- agent_transaction_extractor(text_input, transaction_type, model=None)
  → Extract data using transaction-specific prompt

🤖 LEGACY AGENTS (still available):
- agent_basic_extractor()
- agent_smart_categorizer()
- agent_multi_item_handler()

🛠️ UTILITIES:
- format_amount()
- validate_extraction()

USAGE EXAMPLES:
---------------
# Full flow with intent detection
result = route_with_intent("chai 60 rupees")

# Skip intent if you know the type
result = route_with_intent("petrol 500", transaction_type='expense')

# Just intent detection
intent = agent_intent_detector("chai 60 rupees")

# Just extraction
data = agent_transaction_extractor("chai 60", "ex

In [7]:
"""
TEST: Verify Anthropic API Key
"""
# Get the API key
anthropic_key = get_api_key('anthropic_api_key')

if anthropic_key and 'YOUR' not in anthropic_key:
    print(f"✅ API Key found: {anthropic_key[:20]}...{anthropic_key[-4:]}")
    print(f"✅ Key length: {len(anthropic_key)} characters")
else:
    print("❌ API key not configured properly")

✅ Registry loaded successfully: 8 items found
✅ API Key found: sk-ant-api03-zSEHYFr...1AAA
✅ Key length: 108 characters


In [8]:
"""
TEST: Make a simple Claude API call
"""
from anthropic import Anthropic

# Get client
client = get_anthropic_client()

if client:
    try:
        # Simple test call
        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=50,
            messages=[{"role": "user", "content": "Say 'Hello from Claude!' and nothing else."}]
        )
        
        print("✅ Claude API call successful!")
        print(f"Response: {response.content[0].text}")
    
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Could not initialize client")

NameError: name 'get_anthropic_client' is not defined